<a href="https://colab.research.google.com/github/liskibruh/machineLearning/blob/main/VG16_on_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

In [3]:
class_names = ['0', '1', '2', '3', '4', '5']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (150, 150)

In [6]:

!unzip '/content/train.zip' -d '/content/train'

Archive:  /content/train.zip
   creating: /content/train/train/
   creating: /content/train/train/0/
  inflating: /content/train/train/0/11ma3u.jpg  
  inflating: /content/train/train/0/15dfdg.jpg  
  inflating: /content/train/train/0/1an921.jpg  
  inflating: /content/train/train/0/1bnz0k.jpg  
  inflating: /content/train/train/0/1cei5f.jpg  
  inflating: /content/train/train/0/1gez7a.jpg  
  inflating: /content/train/train/0/1hde0g.jpg  
  inflating: /content/train/train/0/1kdloq.jpg  
  inflating: /content/train/train/0/1lq9qd.jpg  
  inflating: /content/train/train/0/1lrw19.jpg  
  inflating: /content/train/train/0/1o9iki.jpg  
  inflating: /content/train/train/0/1ot3qe.jpg  
  inflating: /content/train/train/0/1p85ki.jpg  
  inflating: /content/train/train/0/1pxizt.jpg  
  inflating: /content/train/train/0/1q9f0m.jpg  
  inflating: /content/train/train/0/1r5547.jpg  
  inflating: /content/train/train/0/1r5ley.jpg  
  inflating: /content/train/train/0/1sd1x9.jpg  
  inflating: /con

In [7]:
!unzip '/content/test.zip' -d '/content/test'

Archive:  /content/test.zip
   creating: /content/test/test/
   creating: /content/test/test/0/
  inflating: /content/test/test/0/1101l4.jpg  
  inflating: /content/test/test/0/13x7qf.jpg  
  inflating: /content/test/test/0/1nsjvc.jpg  
  inflating: /content/test/test/0/1qty0g.jpg  
  inflating: /content/test/test/0/1wnyqp.jpg  
  inflating: /content/test/test/0/1wuqk0.jpg  
  inflating: /content/test/test/0/1zglip.jpg  
  inflating: /content/test/test/0/21f1f9.jpg  
  inflating: /content/test/test/0/25avzz.jpg  
  inflating: /content/test/test/0/26mfm1.jpg  
  inflating: /content/test/test/0/28qgen.jpg  
  inflating: /content/test/test/0/28vtyu.jpg  
  inflating: /content/test/test/0/2lz6ao.jpg  
  inflating: /content/test/test/0/2pyagj.jpg  
  inflating: /content/test/test/0/2r9ku3.jpg  
  inflating: /content/test/test/0/2tzdge.jpg  
  inflating: /content/test/test/0/30dnto.jpg  
  inflating: /content/test/test/0/37efqh.jpg  
  inflating: /content/test/test/0/3cx4aa.jpg  
  inflating

In [10]:
def load_data():
    
    datasets = ['/content/train/train', '/content/test/test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [11]:
(train_images, train_labels), (test_images, test_labels) = load_data()

Loading /content/train/train


100%|██████████| 49/49 [00:00<00:00, 462.70it/s]


Loading /content/test/test


100%|██████████| 14/14 [00:00<00:00, 395.53it/s]


In [12]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)

In [13]:
train_images = train_images / 255.0 
test_images = test_images / 255.0

In [14]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

In [15]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

Epoch 1/20
6/6 [==============================] - 21s 3s/step - loss: 4.0243 - accuracy: 0.2579 - val_loss: 1.5370 - val_accuracy: 0.4000
Epoch 2/20
6/6 [==============================] - 16s 3s/step - loss: 1.6319 - accuracy: 0.3238 - val_loss: 1.6278 - val_accuracy: 0.3029
Epoch 3/20
6/6 [==============================] - 16s 3s/step - loss: 1.5977 - accuracy: 0.3238 - val_loss: 1.5029 - val_accuracy: 0.3029
Epoch 4/20
6/6 [==============================] - 16s 3s/step - loss: 1.4073 - accuracy: 0.3940 - val_loss: 1.4973 - val_accuracy: 0.4114
Epoch 5/20
6/6 [==============================] - 16s 3s/step - loss: 1.3912 - accuracy: 0.4341 - val_loss: 1.4104 - val_accuracy: 0.4571
Epoch 6/20
6/6 [==============================] - 16s 3s/step - loss: 1.3182 - accuracy: 0.4871 - val_loss: 1.3841 - val_accuracy: 0.4629
Epoch 7/20
6/6 [==============================] - 16s 3s/step - loss: 1.2492 - accuracy: 0.5043 - val_loss: 1.3696 - val_accuracy: 0.5029
Epoch 8/20
6/6 [==================

In [17]:
test_loss = model.evaluate(test_images, test_labels)

7/7 [==============================] - 2s 244ms/step - loss: 2.4130 - accuracy: 0.3650
